In [ ]:
import pickle
import tensorflow as tf
import random
from itertools import combinations
import numpy as np
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, Lambda, Conv2D, MaxPool2D
from tensorflow.keras.models import Sequential
from tensorflow.keras import Model
import tensorflow.keras.backend as K
import librosa
from tensorflow.keras.utils import Sequence

In [ ]:
with open('hw4_trs.pkl', 'rb') as pickle_file:
    train_data = pickle.load(pickle_file)
print(train_data.shape)

In [ ]:
with open('hw4_tes.pkl', 'rb') as pickle_file:
    test_data = pickle.load(pickle_file)
print(test_data.shape)

In [ ]:
def create_pos_pairs(speaker, L=45):
    batch = []
    all_pairs = list(combinations(range(10), 2))
    l_pairs = random.sample(all_pairs, L)
    for a,b in l_pairs:
        stft_1 = np.abs(librosa.stft(speaker[a], n_fft=1024, hop_length=512)).T
        stft_2 = np.abs(librosa.stft(speaker[b], n_fft=1024, hop_length=512)).T
        batch.append([stft_1, stft_2])
    return batch

In [ ]:
def create_neg_pairs(pos_sp_num, train_data, L=45):
    batch = []
    
    start = pos_sp_num*10
    end = start + 10     
    pos_spk = train_data[start: end]
    neg_spk = train_data[:start] + train_data[end:]
    neg_sample = random.sample(neg_spk, L)
        
    for l in range(L):
        pos = random.choice(pos_spk)
        
        stft_pos = np.abs(librosa.stft(pos, n_fft=1024, hop_length=512)).T
        stft_neg = np.abs(librosa.stft(neg_sample[l], n_fft=1024, hop_length=512)).T
        batch.append([stft_pos, stft_neg])

    return batch

In [ ]:
mini_batches = []
train_data = list(train_data)
for i in range(50):
    pos_batch = create_pos_pairs(train_data[i:i+10])
    neg_batch = create_neg_pairs(i, train_data)
    mini_batches += pos_batch + neg_batch
#     mini_batches.append(mini_batch)

In [7]:
mini_np = np.stack(mini_batches)

In [8]:
mini_np.shape

(4500, 2, 32, 513)

In [9]:
# Function copied from [1]
def euclidean_dist(vects):
    x, y = vects
    sum_square = K.sum(K.square(x - y), axis=1, keepdims=True)
    return K.sqrt(K.maximum(sum_square, K.epsilon()))

In [10]:
# Function copied from [1]
def contrastive_loss(y_true, y_pred):
    '''Contrastive loss from Hadsell-et-al.'06
    http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    '''
    margin = 1
    square_pred = K.square(y_pred)
    margin_square = K.square(K.maximum(margin - y_pred, 0))
    return K.mean(y_true * square_pred + (1 - y_true) * margin_square)

In [11]:
def siamese_model(input_shape):
    left_input = Input(input_shape)
    right_input = Input(input_shape)
    
    # Base Network
    base_model = Sequential()
    base_model.add(Conv2D(32, kernel_size = (5, 5), input_shape = input_shape, activation = 'relu'))
    base_model.add(MaxPool2D())
    base_model.add(Conv2D(64, kernel_size = (3, 3), activation = 'relu'))
    base_model.add(MaxPool2D())
    base_model.add(Flatten())
    base_model.add(Dense(1000, activation = 'sigmoid'))
    
    left_output = base_model(left_input)
    right_output = base_model(right_input)
   
    L2_distance = Lambda(euclidean_dist, output_shape=(1000, 1))([left_output, right_output])
    
    prediction = Dense(1, activation='sigmoid')(L2_distance)
    
    siamese_model = Model(inputs = [left_input, right_input], outputs = prediction)
    
    return siamese_model

In [12]:
model = siamese_model([32, 513, 1])
model.compile(loss=contrastive_loss, optimizer = tf.keras.optimizers.Adam())

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [13]:
Y = [] 
for i in range(50):
    y = np.zeros(90, dtype = int)
    y[:45] += 1
    Y.append(y)
Y = np.hstack(Y)

In [14]:
mini_np_exp = np.expand_dims(mini_np, axis = 4)

In [15]:
mini_np_exp.shape

(4500, 2, 32, 513, 1)

In [ ]:
model.fit([mini_np_exp[:, 0], mini_np_exp[:, 1]], Y, batch_size=90, epochs=50)

Epoch 1/50
4500/4500 [==============================] - 133s 29ms/sample - loss: 0.1982
Epoch 2/50
4500/4500 [==============================] - 138s 31ms/sample - loss: 0.1743
Epoch 3/50
4500/4500 [==============================] - 135s 30ms/sample - loss: 0.1618
Epoch 4/50
4500/4500 [==============================] - 121s 27ms/sample - loss: 0.1512
Epoch 5/50
4500/4500 [==============================] - 135s 30ms/sample - loss: 0.1456
Epoch 6/50
4500/4500 [==============================] - 140s 31ms/sample - loss: 0.1410
Epoch 7/50
4500/4500 [==============================] - 135s 30ms/sample - loss: 0.1368
Epoch 8/50
4500/4500 [==============================] - 128s 29ms/sample - loss: 0.1330
Epoch 9/50
4500/4500 [==============================] - 114s 25ms/sample - loss: 0.1293
Epoch 10/50
4500/4500 [==============================] - 52s 12ms/sample - loss: 0.1259
Epoch 11/50
4500/4500 [==============================] - 52s 11ms/sample - loss: 0.1227
Epoch 12/50
4500/4500 [=========

## References
1. https://keras.io/examples/mnist_siamese/
2. https://medium.com/predict/face-recognition-from-scratch-using-siamese-networks-and-tensorflow-df03e32f8cd0
3. https://becominghuman.ai/siamese-networks-algorithm-applications-and-pytorch-implementation-4ffa3304c18
4. https://towardsdatascience.com/one-shot-learning-with-siamese-networks-using-keras-17f34e75bb3d

In [ ]:
mini_batches_test = []
test_data = list(test_data)
for i in range(50):
    pos_batch = create_pos_pairs(test_data[i:i+10])
    neg_batch = create_neg_pairs(i, test_data)
    mini_batches_test += pos_batch + neg_batch

In [ ]:
mini_np_test = np.stack(mini_batches_test)

In [ ]:
mini_np_test.shape

In [ ]:
mini_np_exp_test = np.expand_dims(mini_np_test, axis = 4)
mini_np_exp_test.shape

In [ ]:
model.evaluate([mini_np_exp_test[:, 0], mini_np_exp_test[:, 1]] ,Y, batch_size=10)